In [1]:
import pandas as pd
import numpy as np

In [1]:
car = pd.read_csv('/kaggle/input/quicker-car/quikr_car.csv')

In [1]:
car.head()

In [1]:
car.info()

In [1]:
car['year'].unique()

In [1]:
car['Price'].unique()

In [1]:
car['kms_driven'].unique()

In [1]:
car['fuel_type'].unique()

In [1]:
car['name'].unique()

## Dataset issues
- year has many non-year values
- year object to int
- price has string values
- price object to int
- kms_driven object to int
- kms_driven as some nan values
- fuel_type as nan values
- keeping only  names in names

# Cleaning

- year cleaning

In [1]:
backup = car.copy()

In [1]:
car = car[car['year'].str.isnumeric()]


In [1]:
car['year']=car['year'].astype(int)

In [1]:
car['year'].unique()

 - Price entity cleaning

In [1]:
car = car[car['Price']!="Ask For Price"]

In [1]:
car['Price'] = car['Price'].str.replace(',','').astype(int)

In [1]:
car['Price'].head()

 - Kms_driven cleaning

In [1]:
car['kms_driven'] = car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

In [1]:
car = car[car['kms_driven'].str.isnumeric()]

In [1]:
car

In [1]:
car['kms_driven'] = car['kms_driven'].astype(int)

In [1]:
car.info()

- fuel type cleaning

In [1]:
car = car[~car['fuel_type'].isna()]

In [1]:
car.info()

- Name preprocessing

In [1]:
car['name'] = car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [1]:
car.info()

In [1]:
car = car.reset_index(drop = True)

- Removing Highly diviating Values

In [1]:
car.describe()

In [1]:
car = car[car['Price']<5e6].reset_index(drop = True)

In [1]:
car.describe()

In [1]:
car.to_csv('cleaned car.csv')

## Model

In [1]:
x = car.drop(columns = 'Price')

In [1]:
y = car['Price']

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [1]:
ohe = OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

In [1]:
column_trans = make_column_transformer((OneHotEncoder(categories= ohe.categories_),['name','company','fuel_type']), remainder = 'passthrough')

In [1]:
lr=LinearRegression()

In [1]:
pipe = make_pipeline(column_trans, lr)

In [1]:
pipe.fit(X_train,y_train)

In [1]:
y_pred = pipe.predict(X_test)

In [1]:
r2_score(y_test,y_pred)

In [1]:
scores = []
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [1]:
np.argmax(scores)

In [1]:
scores[np.argmax(scores)]

In [1]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

In [1]:
pipe.predict(pd.DataFrame(columns=X_test.columns, data=np.array(['Audi A4 1.8','Audi', 2018,1000,'Petrol']).reshape(1,5)))